## Binary Classification Sonar Project for the Navy - Mines vs. Rocks 

##### Step 1 - Description of the Dataset 
This is a dataset that describes sonar chirp returns bouncing off different services. The 60 input variables are the strength of the returns at different angles. It is a binary classification problem that requires a model to differentiate rocks from metal cylinders

##### Step 2 - We will start off by importing all of the classes and functions we will need:

In [134]:
# Import Libararies

import numpy
import numpy as np
import pandas
from keras.models import Sequential, Model
from keras import layers
from keras.layers import Dense, Input
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras import regularizers

##### Extended Step 2 - Next, we can initialize the random number generator to ensure that we always get the same results when executing this code. This will help if we are debugging:

In [4]:
# fix random seed for reproducibility

seed = 7
numpy.random.seed(seed)

##### Extended Step 2 - Now we can load the dataset using pandas and split the columns into 60 input variables (X) and 1 output variable (Y). We use pandas to load the data because it easily handles strings (the output variable), whereas attempting to load the data directly using NumPy would be more difficult.

In [5]:
# load Sonar dataset using Pandas
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values

X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [6]:
# Shape of Data

dataset.shape

(208, 61)

##### Extended Step 2  - The output variable is string values. We must convert them into integer values 0 and 1.

In [7]:
Y

array(['R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M

##### Extended Step 2 - We can do this using the LabelEncoder class from scikit-learn. This class will model the encoding required using the entire dataset via the fit() function, then apply the encoding to create a new output variable using the transform() function or use both functions in one line using fit_transform() function

In [8]:
# Encoded output 'M' & 'R' into '1', '0'
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(Y)
encoded_Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0], dtype=int64)

##### Extended Step 2 -  Defining the function that creates our baseline model

In [9]:
# baseline model
def create_baseline():
        # create model, write code below
        model = Sequential()
        model.add(Dense(60, activation="relu", input_shape=(60,)))
        model.add(Dense(1, activation="sigmoid"))
          
        # Compile model, write code below
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model


##### Extended Step 2 - Evaluate this model using stratified cross validation in the scikit-learn framework

In [10]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 84.66% (6.36%)


##### Step 3 - Re-Run The Baseline Model With Data Preparation & Evaluate baseline model with standardized dataset

In [11]:
# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 85.59% (7.46%)


##### Step 4.1 - Evaluate a Smaller Network

In [12]:
def create_smaller():
        # create model, write code below
        model = Sequential()
        model.add(Dense(30, activation="relu", input_shape=(60,)))
        model.add(Dense(1, activation="sigmoid"))
          
        # Compile model, write code below
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model

In [13]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=12, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 85.50% (6.30%)


##### Step 4.2 - Evaluate a Larger Network

In [14]:
# larger model
def create_larger():
    # create model
    model = Sequential()
    model.add(Dense(60, activation="relu", input_shape=(60,)))
    model.add(Dense(30, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    # Compile model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [15]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 84.61% (6.36%)


##### Step 5 - Developing a model that overfits

In [135]:
# Overfit model By Keras api
def create_model():
        # create model, write code below
        model = Sequential()
        model.add(Dense(32, activation="relu", input_shape=(60,)))
        
        model.add(Dense(32,activation="relu"))
 
        model.add(Dense(1, activation="sigmoid"))
          
        # Compile model, write code below
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model

In [137]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=100, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Main Model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
186/186 [==============================] - 15s 81ms/step - loss: 0.6041 - acc: 0.6720
Epoch 2/100
186/186 [==============================] - 0s 2ms/step - loss: 0.4820 - acc: 0.7796
Epoch 3/100
186/186 [==============================] - 0s 2ms/step - loss: 0.4076 - acc: 0.8333
Epoch 4/100
186/186 [==============================] - 0s 2ms/step - loss: 0.3419 - acc: 0.8763
Epoch 5/100
186/186 [==============================] - 0s 2ms/step - loss: 0.2892 - acc: 0.9140
Epoch 6/100
186/186 [==============================] - 0s 2ms/step - loss: 0.2465 - acc: 0.9355
Epoch 7/100
186/186 [==============================] - 0s 2ms/step - loss: 0.2061 - acc: 0.9516
Epoch 8/100
186/186 [==============================] - 0s 2ms/step - loss: 0.1684 - acc: 0.9731
Epoch 9/100
186/186 [==============================] - 0s 2ms/step - loss: 0.1392 - acc: 0.9839
Epoch 10/100
186/186 [==============================] - 0s 2ms/step - loss: 0.1144 - acc: 0.9946
Epoch 11/100
186/186 [===============

186/186 [==============================] - 0s 2ms/step - loss: 3.0881e-04 - acc: 1.0000
Epoch 85/100
186/186 [==============================] - 0s 2ms/step - loss: 2.9763e-04 - acc: 1.0000
Epoch 86/100
186/186 [==============================] - 0s 2ms/step - loss: 2.8852e-04 - acc: 1.0000
Epoch 87/100
186/186 [==============================] - 0s 2ms/step - loss: 2.7832e-04 - acc: 1.0000
Epoch 88/100
186/186 [==============================] - 0s 2ms/step - loss: 2.6841e-04 - acc: 1.0000
Epoch 89/100
186/186 [==============================] - 0s 2ms/step - loss: 2.5990e-04 - acc: 1.0000
Epoch 90/100
186/186 [==============================] - 0s 2ms/step - loss: 2.5127e-04 - acc: 1.0000
Epoch 91/100
186/186 [==============================] - 0s 2ms/step - loss: 2.4421e-04 - acc: 1.0000
Epoch 92/100
186/186 [==============================] - 0s 2ms/step - loss: 2.3569e-04 - acc: 1.0000
Epoch 93/100
186/186 [==============================] - 0s 2ms/step - loss: 2.2820e-04 - acc: 1.0000
Epo

187/187 [==============================] - 1s 3ms/step - loss: 6.0116e-04 - acc: 1.0000
Epoch 67/100
187/187 [==============================] - 0s 2ms/step - loss: 5.7936e-04 - acc: 1.0000
Epoch 68/100
187/187 [==============================] - 0s 2ms/step - loss: 5.5201e-04 - acc: 1.0000
Epoch 69/100
187/187 [==============================] - 0s 2ms/step - loss: 5.3086e-04 - acc: 1.0000
Epoch 70/100
187/187 [==============================] - 0s 2ms/step - loss: 5.0796e-04 - acc: 1.0000
Epoch 71/100
187/187 [==============================] - 0s 2ms/step - loss: 4.9081e-04 - acc: 1.0000
Epoch 72/100
187/187 [==============================] - 0s 3ms/step - loss: 4.6907e-04 - acc: 1.0000
Epoch 73/100
187/187 [==============================] - 0s 2ms/step - loss: 4.5264e-04 - acc: 1.0000
Epoch 74/100
187/187 [==============================] - 0s 2ms/step - loss: 4.3345e-04 - acc: 1.0000
Epoch 75/100
187/187 [==============================] - 0s 2ms/step - loss: 4.1889e-04 - acc: 1.0000
Epo

187/187 [==============================] - 0s 2ms/step - loss: 0.0012 - acc: 1.0000
Epoch 49/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0012 - acc: 1.0000
Epoch 50/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0011 - acc: 1.0000
Epoch 51/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0010 - acc: 1.0000
Epoch 52/100
187/187 [==============================] - 0s 2ms/step - loss: 9.6648e-04 - acc: 1.0000
Epoch 53/100
187/187 [==============================] - 0s 2ms/step - loss: 9.3204e-04 - acc: 1.0000
Epoch 54/100
187/187 [==============================] - 0s 2ms/step - loss: 8.7649e-04 - acc: 1.0000
Epoch 55/100
187/187 [==============================] - 0s 2ms/step - loss: 8.3342e-04 - acc: 1.0000
Epoch 56/100
187/187 [==============================] - 0s 2ms/step - loss: 7.9154e-04 - acc: 1.0000
Epoch 57/100
187/187 [==============================] - 0s 2ms/step - loss: 7.5076e-04 - acc: 1.0000s - loss: 9.1971e-04

187/187 [==============================] - 0s 2ms/step - loss: 0.0075 - acc: 1.0000
Epoch 30/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0067 - acc: 1.0000
Epoch 31/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0062 - acc: 1.0000
Epoch 32/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0056 - acc: 1.0000A: 0s - loss: 0.0059 - acc: 1.
Epoch 33/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0051 - acc: 1.0000
Epoch 34/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0047 - acc: 1.0000
Epoch 35/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0042 - acc: 1.0000
Epoch 36/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0039 - acc: 1.0000
Epoch 37/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0036 - acc: 1.0000
Epoch 38/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0033 - acc: 1.0000
Epoch 39/100


...

Epoch 58/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0010 - acc: 1.0000
Epoch 59/100
187/187 [==============================] - 0s 2ms/step - loss: 9.6713e-04 - acc: 1.0000
Epoch 60/100
187/187 [==============================] - 0s 2ms/step - loss: 9.1876e-04 - acc: 1.0000
Epoch 61/100
187/187 [==============================] - 0s 2ms/step - loss: 8.7649e-04 - acc: 1.0000
Epoch 62/100
187/187 [==============================] - 0s 2ms/step - loss: 8.3547e-04 - acc: 1.0000
Epoch 63/100
187/187 [==============================] - 0s 2ms/step - loss: 7.9405e-04 - acc: 1.0000
Epoch 64/100
187/187 [==============================] - 0s 2ms/step - loss: 7.6046e-04 - acc: 1.0000
Epoch 65/100
187/187 [==============================] - 0s 2ms/step - loss: 7.2487e-04 - acc: 1.0000
Epoch 66/100
187/187 [==============================] - 0s 2ms/step - loss: 6.9612e-04 - acc: 1.0000
Epoch 67/100
187/187 [==============================] - 0s 2ms/step - loss: 6.6677e-04 - acc: 1

188/188 [==============================] - 0s 2ms/step - loss: 0.0024 - acc: 1.0000
Epoch 40/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0022 - acc: 1.0000
Epoch 41/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0021 - acc: 1.0000
Epoch 42/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0019 - acc: 1.0000
Epoch 43/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0018 - acc: 1.0000
Epoch 44/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0017 - acc: 1.0000
Epoch 45/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0016 - acc: 1.0000
Epoch 46/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0015 - acc: 1.0000
Epoch 47/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0014 - acc: 1.0000
Epoch 48/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0013 - acc: 1.0000
Epoch 49/100
188/188 [=====================

188/188 [==============================] - 0s 2ms/step - loss: 0.0127 - acc: 1.0000
Epoch 22/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0108 - acc: 1.0000
Epoch 23/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0096 - acc: 1.0000
Epoch 24/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0083 - acc: 1.0000
Epoch 25/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0073 - acc: 1.0000
Epoch 26/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0066 - acc: 1.0000
Epoch 27/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0058 - acc: 1.0000
Epoch 28/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0053 - acc: 1.0000
Epoch 29/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0048 - acc: 1.0000
Epoch 30/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0043 - acc: 1.0000
Epoch 31/100
188/188 [=====================

Epoch 3/100
188/188 [==============================] - 0s 2ms/step - loss: 0.4384 - acc: 0.8404
Epoch 4/100
188/188 [==============================] - 0s 2ms/step - loss: 0.3598 - acc: 0.8830
Epoch 5/100
188/188 [==============================] - 0s 2ms/step - loss: 0.2931 - acc: 0.9149
Epoch 6/100
188/188 [==============================] - 0s 2ms/step - loss: 0.2416 - acc: 0.9309
Epoch 7/100
188/188 [==============================] - 0s 2ms/step - loss: 0.2041 - acc: 0.9415
Epoch 8/100
188/188 [==============================] - 0s 2ms/step - loss: 0.1677 - acc: 0.9574
Epoch 9/100
188/188 [==============================] - 0s 2ms/step - loss: 0.1381 - acc: 0.9734
Epoch 10/100
188/188 [==============================] - 0s 2ms/step - loss: 0.1106 - acc: 0.9734
Epoch 11/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0910 - acc: 0.9894
Epoch 12/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0770 - acc: 1.0000
Epoch 13/100
188/188 [===============

188/188 [==============================] - 0s 2ms/step - loss: 2.0443e-04 - acc: 1.0000
Epoch 87/100
188/188 [==============================] - 0s 2ms/step - loss: 1.9804e-04 - acc: 1.0000
Epoch 88/100
188/188 [==============================] - 0s 2ms/step - loss: 1.9124e-04 - acc: 1.0000
Epoch 89/100
188/188 [==============================] - 0s 2ms/step - loss: 1.8508e-04 - acc: 1.0000
Epoch 90/100
188/188 [==============================] - 0s 2ms/step - loss: 1.7877e-04 - acc: 1.0000
Epoch 91/100
188/188 [==============================] - 0s 2ms/step - loss: 1.7234e-04 - acc: 1.0000
Epoch 92/100
188/188 [==============================] - 0s 2ms/step - loss: 1.6684e-04 - acc: 1.0000
Epoch 93/100
188/188 [==============================] - 0s 2ms/step - loss: 1.6104e-04 - acc: 1.0000
Epoch 94/100
188/188 [==============================] - 0s 2ms/step - loss: 1.5596e-04 - acc: 1.0000
Epoch 95/100
188/188 [==============================] - 0s 2ms/step - loss: 1.5046e-04 - acc: 1.0000
Epo

####### Model overfit after 12 epochs

##### Step 6 -Tunning the model

In [16]:
# Main model By Keras api
def create_model():
        # create model, write code below
        model = Sequential()
        model.add(Dense(32, activation="relu", input_shape=(60,)))
        
        model.add(Dense(32,activation="relu"))
 
        model.add(Dense(1, activation="sigmoid"))
          
        # Compile model, write code below
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model

In [17]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=12, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Main Model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Main Model: 86.52% (6.51%)


##### Step 7 -  Rewriting the code using the Keras Functional API

In [18]:
# Main model By Keras api
def create_model_by_function_api():
    # create model, write code below
    inputs = Input(shape=(60,))
    x_input = Dense(32, activation='relu')(inputs)
    x_input = Dense(32, activation='relu')(x_input)
    predictions = Dense(1, activation = 'sigmoid')(x_input)
    model = Model(inputs=inputs, outputs=predictions)
        
    # Compile model, write code below
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [19]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model_by_function_api, epochs=12, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Functional Api Model : %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Functional Api Model : 86.52% (6.51%)


##### Step 8 - Rewriting the code by doing Model Subclassing

In [31]:

class SubClass_Model(tf.keras.Model):
    #inputs = Input(shape=(60,))
    def __init__(self):
        super(SubClass_Model, self).__init__()
        self.dense1 = tf.keras.layers.Dense(30, activation='relu')
        self.dense2 = tf.keras.layers.Dense(30, activation='relu')
        self.dense3 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x_input = self.dense1(inputs)
        x_input = self.dense2(x_input)
        outputs = self.dense3(x_input)
        return outputs

In [32]:
# Main model By Keras api
def create_model_by_SubClass():
    # create model, write code below
    model = SubClass_Model()
        
    # Compile model, write code below
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [34]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model_by_SubClass, epochs=12, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Sub Class Api Model : %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Sub Class Api Model : 84.02% (7.04%)


##### Step 9 -  Rewriting the code without using scikit-learn

In [133]:
numpy.random.seed(seed)
data=dataset
k = 10
num_validation_samples = len(data) // k
np.random.shuffle(data)
validation_scores = []
for fold in range(k):
    validation_inputs = X[num_validation_samples * fold:num_validation_samples * (fold + 1)]
    training_inputs = np.append(X[:num_validation_samples * fold], X[num_validation_samples * (fold + 1):], axis=0) #, X[num_validation_samples * (fold + 1):]
    
    validation_targets = encoded_Y[num_validation_samples * fold:num_validation_samples * (fold + 1)]
    training_targets = np.append(encoded_Y[:num_validation_samples * fold] , encoded_Y[num_validation_samples * (fold + 1):], axis=0)

    model = create_model() 
    validation_scores = model.fit(training_inputs,training_targets,epochs=12,batch_size=5,verbose=0, validation_data=(validation_inputs,validation_targets))
    #history_dict = validation_score.history
    #validation_scores.append(np.average(history_dict['val_acc']))
    
history_dict = validation_scores.history
validation_score = np.average(history_dict['val_acc'])
#validation_score = np.average(validation_scores)
print("Model using K-Fold without skitlearn : %.2f%% " % (validation_score*100))

Model using K-Fold without skitlearn : 88.75% 
